In [1]:
import os, sys, time
from time import sleep
from pathlib import Path
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from pandas import Series, DataFrame
import pyodbc
from openpyxl import Workbook, load_workbook
from openpyxl.drawing.image import Image
from openpyxl.utils.dataframe import dataframe_to_rows
import tempfile
from ttkthemes import ThemedStyle
import tkinter as tk
import tkinter.ttk as ttk
from tkinter import messagebox, filedialog, commondialog

In [2]:
class QC_Metrics_AvgTurnAround(): # {
    
    user_name = str(os.getlogin())
    outbound_dir = "C:/data/outbound/" + str(pd.Timestamp.now())[:10]
    desktop_dir = "OneDrive - Agilent Technologies/Desktop"
    
    def __init__(self, the_logger): # {
        self.the_logger = the_logger
        # Get/Set USERNAME & DESKTOP DIRECTORIES
        self.user_name_dir = os.path.join("C:/Users/", self.user_name)
        self.desktop_dir = os.path.join(self.user_name_dir, self.desktop_dir)
        print(self.user_name_dir)
        print(self.desktop_dir)
        self.run(day_range=30)
    # }
    
    def run(self, day_range): # {
        # TRY THE FOLLOWING
        try: # {
            # [2020-02-28]\\self.time_unit = time_unit
            # [2020-02-28]\\self.time_value = time_value
            self.day_range = day_range
            # get/set current date variable
            # [2020-03-06]\\the_date = pd.Timestamp.now()
            # [2020-03-11]\\the_date = pd.Timestamp(ts_input=str(self.end_date.get()))
            the_date = pd.Timestamp.now()
            # create variable for a month ago
            one_month_ago = the_date - timedelta(days = int(day_range))
            # [2020-02-28]\\
            ##one_month_ago = the_date - pd.Timedelta(unit=str(self.time_unit), value=str(self.time_value))
            print(type(one_month_ago))
            # CREATE METRICS TABLE FROM CLASS METHOD
            self.df_metrics_table = self.create_metrics_table()
            #############################
            # create .csv with no drops #
            #############################
            """
            self.df_metrics_table.to_csv(os.path.join(self.outbound_dir, "df_metrics_noDrop.csv"), 
                                         index=True)
            # DROP ROWS WITH PRODUCT LEVEL && QCDATE = NONE
            self.df_metrics_table.dropna(axis=0, subset=['QCDate', 'ProductLevel'], how='any', 
                                         inplace=True)
            # create .csv with DROPS
            self.df_metrics_table.to_csv(os.path.join(self.outbound_dir, "df_metrics_DROP.csv"), 
                                         index=True)
            """
            # Set index of METRICS
            # [2020-03-11]\\self.df_metrics_table.set_index(['QCDate', 'ProductLevel'], inplace=True)
            """
            SET INDEX TO QCDAte
            """
            # [2020-03-20]\\self.df_metrics_table.index = self.df_metrics_table['QCDate']
            # [2020-03-20]\\del self.df_metrics_table['QCDate']
            # [2020-03-11]\\del self.df_metrics_table['ProductLevel']
            print(self.df_metrics_table)
            # Display Index
            print(self.df_metrics_table.index)
            #############################
            # create .csv with UNSORTED #
            #############################
            self.df_metrics_table.to_csv(os.path.join(self.outbound_dir, "df_QC_metrics_UNSORTED.csv"), 
                                         index=True)
            # SORT INDEX
            self.df_metrics_table.sort_index(inplace=True)
            ###########################
            # create .csv with SORTED #
            ###########################
            self.df_metrics_table.to_csv(os.path.join(self.outbound_dir, "df_QC_metrics_SORTED.csv"), 
                                         index=True)
            print("SORTED:\n")
            print(self.df_metrics_table.index)
            # SLICE & DICE THE DATAFRAME
            # [2020-03-11]\\self.df_level_1s = self.df_metrics_table.loc[('2017-01-02', 1.0)]
            # [2020-03-11]\\print(self.df_metrics_table.loc[('2017-01-02', '1')])
            print("<<<<< df_level_1s >>>>\n" + str(self.df_metrics_table.loc['2020-01-01':'2020-03-19'].describe()))
            df_random_range = self.df_metrics_table.loc['2019-01-01':'2020-03-01']
            df_random_range.to_csv(os.path.join(self.outbound_dir, 'df-random-range-'
                                                + str(pd.Timestamp.now())[:10]
                                                + ".csv"), index=True)
            """
            # GROUP THE DATA BY DAY, AND TAKE THE MEAN FOR EACH GROUP (i.e each day)
            df_random_range_d_mean = df_random_range.resample('D').mean()
            df_random_range_d_mean.to_csv(os.path.join(self.outbound_dir, "df-random-range-d-mean-" 
                                                      + str(pd.Timestamp.now())[:10]
                                                      + ".csv"), index=True)
            # GROUP THE DATA BY DAY, AND TAKE THE SUM FOR EACH GROUP (i.e. each day)
            df_random_range_d_sum = df_random_range.resample('D').sum()
            df_random_range_d_mean.to_csv(os.path.join(self.outbound_dir, 'df-random-range-d-sum-'
                                                      + str(pd.Timestamp.now())[:10]
                                                      + ".csv"), index=True)
            """
            # CHECK OUT COLUMNS
            # [2020-03-11]\\print("COLUMNS:\n" + str(self.df_level_1s.columns))
            # [2020-03-11]\\print(self.df_level_1s)
            """
            { PERFORM GROUPBY HERE }
            { PERFORM GROUPBY HERE }
            """
            # how many PfBatchID, ProductLevels, and Products, are they each month?
            df_groupby_test_1 = df_random_range.groupby(['PfBatchID', 'ProductLevel'], as_index=False)['QCDate'].count()
            df_groupby_test_1.to_csv(os.path.join(self.outbound_dir, 'df-groupby-test-1-'
                                                 + str(pd.Timestamp.now())[:10]
                                                 + ".csv"), index=True)
            df_groupby_test_2 = df_random_range.groupby('QCDate', as_index=True).agg({"ProductLevel":"sum"})
            df_groupby_test_2.to_csv(os.path.join(self.outbound_dir, "df-groupby-test-2-"
                                                + str(pd.Timestamp.now())[:10]
                                                + ".csv"), index=True)
            # GROUP BY ["ProductLevel"]
            df_by_product_level = df_random_range.groupby(['QCDate', 'ProductLevel'])['PfBatchID'].count()
            # [2020-03-20]\\df_by_product_level = df_random_range.groupby('ProductLevel')
            # PRINT GROUPS
            print(df_by_product_level.groups)
            # CHECK OUT THE TYPE OF OBJECT
            print("TYPE == " + str(type(df_by_product_level)))
            # SUMARY STATS OVER THE LEVELS
            print(df_by_product_level.describe().head())
            # CAST GROUPING as a LIST AND CHECK OUT ONE LEVEL
            print("level-listing #1 :\n" + str(list(df_by_product_level)[1]))
            level_1 = pd.DataFrame(data=list(df_by_product_level)[1]) # , dtype=np.str, index=['QCDate'])
            level_2 = pd.DataFrame(data=list(df_by_product_level)[2]) #, dtype=np.str, index=['QCDate'])
            level_3 = pd.DataFrame(data=list(df_by_product_level)[3]) #, dtype=np.str, index=['QCDate'])
            # EXPORT ??
            level_1.to_csv(os.path.join(self.outbound_dir, 'df-level-1-'
                                        + str(pd.Timestamp.now())[:10]
                                        + ".csv"), index=True)
            level_2.to_csv(os.path.join(self.outbound_dir, 'df-level-2-'
                                        + str(pd.Timestamp.now())[:10]
                                        + ".csv"), index=True)
            level_3.to_csv(os.path.join(self.outbound_dir, 'df-level-3-'
                                        + str(pd.Timestamp.now())[:10]
                                        + ".csv"), index=True)
            """
            # GET MEDIAN VALUES BY PRODUCT LEVEL AND PRINT FIRST 5 ROWS
            df_med_by_product_level = df_by_product_level.median()
            df_med_by_product_level.head()
            # Slice out PfBatchID and plot?
            df_id_by_p_level = df_med_by_product_level['PfBatchID']
            df_id_by_p_level.to_csv(os.path.join(self.outbound_dir, 'df-id-by-p-level-'
                                                 + str(pd.Timestamp.now())[:10]
                                                 + '.csv'), index=False)
            """
            # AGGREGATE INTO DAYS BY SUMMING UP VALUES OF EACH HOURLY OBSERVATION
            df_by_product_level.resample('D').sum().to_csv(os.path.join(self.outbound_dir, 'df-product-level-samples-'
                                                                        + str(pd.Timestamp.now())[:10]
                                                                        + ".csv"), index=True)
        # }
        except: # {
            errorMessage = str(sys.exc_info()[0]) + "\n"
            errorMessage = errorMessage + str(sys.exc_info()[1]) + "\n\t\t"
            errorMessage = errorMessage + str(sys.exc_info()[2]) + "\n"
            exc_type, exc_obj, exc_tb = sys.exc_info()
            fname = os.path.split(exc_tb.tb_frame.f_code.co_filename)[1]
            typeE = str("TYPE : " + str(exc_type))
            fileE = str("FILE : " + str(fname))
            lineE = str("LINE : " + str(exc_tb.tb_lineno))
            messageE = str("MESG : " + "\n\n" + str(errorMessage) + "\n")
            print("\n" + typeE + 
                  "\n" + fileE + 
                  "\n" + lineE + 
                  "\n" + messageE)
        # }
        else: # {
            print("Operation Completed Successfully...")
        # }
    # }
    
    """
    Referred to as "ProdflowII" in SQL-Server
    """
    def pull_ProdflowII_table(self, table_name): # {
        # TRY THE FOLLOWING
        try: # {
            # CREATION CONNECTION STR
            conn_str = str(
                r'DRIVER={ODBC Driver 17 for SQL Server};'
                r'SERVER=wtkngappflow1.is.agilent.net;'
                r'DATABASE=ProdFlowII_Prod;'
                r'Trusted_Connection=yes;'
            )
            # CREATE PYODBC CONNECTION
            cnxn_ProdflowII = pyodbc.connect(conn_str)
            # [2020-02-028]\\crsr_ProdflowII = cnxn_ProdflowII.cursor()
            # PERFORM SQL QUERY AND SET AS DATAFRAME
            df_ProdflowII_table = pd.read_sql_query(sql='SELECT * FROM ' + str(table_name),
                                                    con=cnxn_ProdflowII
                                                    )
        # }
        except: # {
            errorMessage = str(sys.exc_info()[0]) + "\n"
            errorMessage = errorMessage + str(sys.exc_info()[1]) + "\n\t\t"
            errorMessage = errorMessage + str(sys.exc_info()[2]) + "\n"
            exc_type, exc_obj, exc_tb = sys.exc_info()
            fname = os.path.split(exc_tb.tb_frame.f_code.co_filename)[1]
            typeE = str("TYPE : " + str(exc_type))
            fileE = str("FILE : " + str(fname))
            lineE = str("LINE : " + str(exc_tb.tb_lineno))
            messageE = str("MESG : " + "\n\n" + str(errorMessage) + "\n")
            print("\n" + typeE + 
                  "\n" + fileE + 
                  "\n" + lineE + 
                  "\n" + messageE)
        # }
        else: # {
            print("Operation Completed Successfully...")
            return df_ProdflowII_table
        # }
    # }
    
    """
    Referred to as "Prodflow" in SQL-Server
    """
    def pull_ProdflowIII_table(self, table_name): # {
        # TRY THE FOLLOWING
        try: # {
            # CREATE CONNECTION STRING
            conn_str = str(
                r'DRIVER={ODBC Driver 17 for SQL Server};'
                r'SERVER=wtkngappflow1.is.agilent.net;'
                r'DATABASE=ProdFlow;'
                r'Trusted_Connection=yes;'
            )
            # CREATE PYODBC CONNECTION
            cnxn_ProdflowIII = pyodbc.connect(conn_str)
            # [2020-02-28]\\crsr_ProdflowIII = cnxn_ProdflowIII,cursor()
            # PERFORM SQL QUERY AND SET AS DATAFRAME
            df_ProdflowIII_table = pd.read_sql_query(sql='SELECT * FROM ' + str(table_name),
                                                     con=cnxn_ProdflowIII,
                                                     parse_dates=['QCDate']
                                                     )
        # }
        except: # {
            errorMessage = str(sys.exc_info()[0]) + "\n"
            errorMessage = errorMessage + str(sys.exc_info()[1]) + "\n\t\t"
            errorMessage = errorMessage + str(sys.exc_info()[2]) + "\n"
            exc_type, exc_obj, exc_tb = sys.exc_info()
            fname = os.path.split(exc_tb.tb_frame.f_code.co_filename)[1]
            typeE = str("TYPE : " + str(exc_type))
            fileE = str("FILE : " + str(fname))
            lineE = str("LINE : " + str(exc_tb.tb_lineno))
            messageE = str("MESG : " + "\n\n" + str(errorMessage) + "\n")
            print("\n" + typeE + 
                  "\n" + fileE + 
                  "\n" + lineE + 
                  "\n" + messageE)
        # }
        else: # {
            print("Operation Completed Successfully...")
            return df_ProdflowIII_table
        # }
    # }
    
    def create_metrics_table(self): # {
        # TRY THE FOLLOWING
        try: # {
            # pull PRODUCTS table
            self.df_products = self.pull_ProdflowII_table(table_name='Products')
            # RENAME ['Product#'] column to ["ProductNo"]
            self.df_products.rename(columns={'Product#': 'ProductNo'}, inplace=True)
            self.df_products.to_csv(os.path.join(self.outbound_dir, "df-PRODUCTS-table-" 
                                                + str(pd.Timestamp.now())[:10]
                                                + ".csv"), index=False)
            print("\tORDERS-TABLE:\n" + str(self.df_products.info()))
            # pull tblProdflow TABLE
            self.df_tblProdflow = self.pull_ProdflowIII_table(table_name='tblProdflow')
            self.df_tblProdflow.to_csv(os.path.join(self.outbound_dir, "df-tblProdflow-"
                                                    + str(pd.Timestamp.now())[:10] 
                                                    + ".csv"), index=False)
            print(self.df_tblProdflow.info())
            ########################
            # CREATE METRICS TABLE #
            ########################
            df_metrics_table = pd.merge(self.df_products, self.df_tblProdflow, on='ProductNo', how='right')
            ######################
            # CHANGE DATA TYPES: #
            ######################
            # CHANGE ['QCDate'] COLUMN TO DATETIME
            df_metrics_table['QCDate'] = pd.to_datetime(df_metrics_table['QCDate'])
            # CHANGE ['ProductLevel'] COLUMN TO FLOAT
            # [2020-03-11]\\df_metrics_table['ProductLevel'] = df_metrics_table['ProductLevel'].values.astype(float)
            # [2020-03-11]\\df_metrics_table.ProductLevel = df_metrics_table.astype(float)
            # [2020-03-11]\\df_metrics_table['ProductLevel'] = df_metrics_table['ProductLevel'].values.astype(int)
            """
            # CHANGE 'ProductLevel' to CATEGORICAL
            df_metrics_table['ProductLevel'] = pd.Categorical(df_metrics_table['ProductLevel'],
                                                          categories=[1, 2, 3], ordered=False)
            """
            #####################################################
            # DROP ALL ROWS WITHOUT A 'QCDATE' & 'ProductLevel' #
            #####################################################
            df_metrics_table.dropna(axis=0, subset=['QCDate', 'ProductLevel'], 
                                    how='any', inplace=True)
            df_metrics_table.to_csv(os.path.join(self.outbound_dir, "df-QC-metrics-table-"
                                                 + str(pd.Timestamp.now())[:10]
                                                 + ".csv"), index=True)
        # }
        except: # {
            errorMessage = str(sys.exc_info()[0]) + "\n"
            errorMessage = errorMessage + str(sys.exc_info()[1]) + "\n\t\t"
            errorMessage = errorMessage + str(sys.exc_info()[2]) + "\n"
            exc_type, exc_obj, exc_tb = sys.exc_info()
            fname = os.path.split(exc_tb.tb_frame.f_code.co_filename)[1]
            typeE = str("TYPE : " + str(exc_type))
            fileE = str("FILE : " + str(fname))
            lineE = str("LINE : " + str(exc_tb.tb_lineno))
            messageE = str("MESG : " + "\n\n" + str(errorMessage) + "\n")
            print("\n" + typeE + 
                  "\n" + fileE + 
                  "\n" + lineE + 
                  "\n" + messageE)
        # }
        else: # {
            print("Operation Completed Successfully...")
            return df_metrics_table
        # }
    # }
    
# }

In [3]:
if __name__ == "__main__": # {
    test_metrics = QC_Metrics_AvgTurnAround(the_logger=None)
# }

C:/Users/derbates
C:/Users/derbates\OneDrive - Agilent Technologies/Desktop
<class 'pandas._libs.tslibs.timestamps.Timestamp'>
Operation Completed Successfully...
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 108535 entries, 0 to 108534
Data columns (total 42 columns):
ProductID          108535 non-null int64
ProductNo          108535 non-null object
ProductName        108507 non-null object
LineID             108535 non-null int64
Description        5976 non-null object
Specials           2443 non-null object
OriginationDate    108141 non-null datetime64[ns]
EntryDate          108533 non-null datetime64[ns]
Status             6036 non-null object
MatrixNumber       107650 non-null object
Components         108535 non-null int64
Availability       4843 non-null object
EquivalentSoln     103091 non-null object
EquivalentNeat     103249 non-null object
Packaging          107581 non-null object
Storage            100889 non-null object
ExpirationDate     108510 non-null float64
ShipTy